**대규모 텐서플로 모델 훈련과 배포**

# 설정

In [1]:
from tensorflow import config

config.list_physical_devices('GPU')

2025-07-31 04:56:43.207147: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753937803.345793      32 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753937803.387365      32 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753937803.713365      32 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753937803.713395      32 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753937803.713397      32 computation_placer.cc:177] computation placer alr

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# 텐서플로 모델 서빙하기

먼저 TF 서빙을 사용하여 모델을 배포한 다음 Google Vertex AI에 배포해 본다.

## 텐서플로 서빙 사용

가장 먼저 모델을 빌드하고 학습한 다음 SavedModel 포맷으로 내보낸다.

### SavedModel 내보내기

MNIST 데이터 세트를 로드하고, 스케일을 조정하고, 분할한다.

In [2]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend, layers
from tensorflow import keras
import tensorflow as tf
from pathlib import Path

mnist_ = mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist_
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
backend.clear_session()
model = keras.Sequential()
model.add(keras.Input((28, 28), dtype=tf.uint8))
model.add(layers.Flatten())
model.add(layers.Rescaling(1 / 255))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
model_name = 'my_mnist_model'
model_version = '0001'
model_path = Path(model_name) / model_version
model.export(model_path)

I0000 00:00:1753937807.694616      32 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5227 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Ti, pci bus id: 0000:06:00.0, compute capability: 8.6


Epoch 1/10


I0000 00:00:1753937809.880152      99 service.cc:152] XLA service 0x7278540021f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753937809.880190      99 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Ti, Compute Capability 8.6
2025-07-31 04:56:49.916035: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1753937809.982879      99 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-07-31 04:56:50.688679: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36_0', 92 bytes spill stores, 96 bytes spill loads

2025-07-31 04:56:50.815757: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36

  78/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2752 - loss: 2.1646

I0000 00:00:1753937812.847531      99 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7287 - loss: 1.0524

2025-07-31 04:57:00.764912: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_34', 296 bytes spill stores, 296 bytes spill loads

2025-07-31 04:57:00.772395: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_34', 8 bytes spill stores, 8 bytes spill loads

2025-07-31 04:57:00.911778: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_34', 8 bytes spill stores, 8 bytes spill loads

2025-07-31 04:57:00.959398: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_34', 792 bytes spill stores, 844 bytes spill loads

2025-07-31 04:57:01.153279: I external/local_xla

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.7287 - loss: 1.0521 - val_accuracy: 0.9034 - val_loss: 0.3727
Epoch 2/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8978 - loss: 0.3722 - val_accuracy: 0.9166 - val_loss: 0.3058
Epoch 3/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9118 - loss: 0.3123 - val_accuracy: 0.9220 - val_loss: 0.2702
Epoch 4/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9219 - loss: 0.2746 - val_accuracy: 0.9294 - val_loss: 0.2472
Epoch 5/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9287 - loss: 0.2484 - val_accuracy: 0.9346 - val_loss: 0.2300
Epoch 6/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9345 - loss: 0.2345 - val_accuracy: 0.9384 - val_loss: 0.2148
Epoch 7/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9391 - loss: 0.2179 - val_accuracy: 0.9430 - val_loss: 0.2017
Epoch 8/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9445 - loss: 0.1995 - val_accura

INFO:tensorflow:Assets written to: my_mnist_model/0001/assets


Saved artifact at 'my_mnist_model/0001'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.uint8, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  125864717775696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  125864717776848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  125864717774160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  125864717777424: TensorSpec(shape=(), dtype=tf.resource, name=None)


파일 트리를 살핀다:

In [3]:
sorted([str(path) for path in model_path.parent.glob('**/*')])

['my_mnist_model/0001',
 'my_mnist_model/0001/assets',
 'my_mnist_model/0001/fingerprint.pb',
 'my_mnist_model/0001/saved_model.pb',
 'my_mnist_model/0001/variables',
 'my_mnist_model/0001/variables/variables.data-00000-of-00001',
 'my_mnist_model/0001/variables/variables.index']

SavedModel을 검사한다:

In [4]:
!saved_model_cli show --dir '{model_path}'

2025-07-31 04:57:38.147831: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753937858.160825     915 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753937858.164828     915 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753937858.177485     915 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753937858.177538     915 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753937858.177546     915 computation_placer.cc:177] computation placer alr

In [5]:
!saved_model_cli show --dir '{model_path}' --tag_set serve

2025-07-31 04:57:40.919984: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753937860.933043     931 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753937860.937065     931 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753937860.948639     931 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753937860.948675     931 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753937860.948682     931 computation_placer.cc:177] computation placer alr

In [6]:
!saved_model_cli show --dir '{model_path}' --tag_set serve --signature_def serving_default

2025-07-31 04:57:43.464925: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753937863.478529     948 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753937863.482575     948 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753937863.494314     948 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753937863.494358     948 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753937863.494365     948 computation_placer.cc:177] computation placer alr

더 자세한 내용을 보려면 다음 명령을 실행한다:
```ipython
!saved_model_cli show --dir '{model_path}' --all
```

### 텐서플로 서빙 설치 및 시작하기

도커를 사용해 TF 서빙을 설치하려면 먼저 [Docker](https://docs.docker.com/install/)가 설치되어 있는지 확인한 후 터미널에서 다음 명령을 실행한다.
```bash
docker pull tensorflow/serving:latest-gpu

docker run -e MODEL_NAME=my_mnist_model --gpus all -it -p 8500:8500 -p 8501:8501 -v path\to\my_mnist_model:/models/my_mnist_model tensorflow/serving:latest-gpu
```

### REST API로 TF 서빙에 쿼리하기

다음으로 TF 서빙에 REST 쿼리를 전송한다:

In [7]:
import json

X_new = X_test[:3]  # 분류할 새로운 숫자 이미지가 3개 있다고 가정한다.
request_json = json.dumps({'signature_name': 'serving_default', 'instances': X_new.tolist()})

In [8]:
request_json[:100] + '...' + request_json[-10:]

'{"signature_name": "serving_default", "instances": [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..., 0, 0]]]}'

이제 텐서플로 서빙의 REST API를 사용하여 예측한다:

In [9]:
import requests

server_url = 'http://172.17.0.3:8501/v1/models/my_mnist_model:predict'
response = requests.post(server_url, request_json)
response.raise_for_status()  # 오류 발생 시 예외 발생
response = response.json()

In [10]:
import numpy as np

y_proba = np.array(response['predictions'])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.98, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.98, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]])

### gRPC API로 TF 서빙에 쿼리하기

In [11]:
from tensorflow_serving.apis.predict_pb2 import PredictRequest

request = PredictRequest()
request.model_spec.name = model_name
request.model_spec.signature_name = 'serving_default'
input_name = 'keras_tensor'
request.inputs[input_name].CopyFrom(tf.make_tensor_proto(X_new))

In [12]:
import grpc
from tensorflow_serving.apis import prediction_service_pb2_grpc

channel = grpc.insecure_channel('172.17.0.3:8500')
predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)
response = predict_service.Predict(request, timeout=10.0)

응답을 텐서로 변환한다:

In [13]:
output_name = 'output_0'
outputs_proto = response.outputs[output_name]
y_proba = tf.make_ndarray(outputs_proto)

In [14]:
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.98, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.98, 0.01, 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  ]],
      dtype=float32)

### 새 모델 버전 배포

In [27]:
# 새로운 MNIST 모델 버전 빌드 및 훈련
model = keras.Sequential()
model.add(keras.Input((28, 28), dtype=tf.uint8))
model.add(layers.Flatten())
model.add(layers.Rescaling(1 / 255))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10


2025-07-31 05:07:48.352345: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 272 bytes spill stores, 272 bytes spill loads

2025-07-31 05:07:48.648933: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 940 bytes spill stores, 940 bytes spill loads

2025-07-31 05:07:48.810329: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 944 bytes spill stores, 944 bytes spill loads

2025-07-31 05:07:48.986006: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 1180 bytes spill stores, 1240 bytes spill loads

2025-07-31 05:07:49.674921: I external

1717/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6483 - loss: 1.2094

2025-07-31 05:07:55.032525: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 272 bytes spill stores, 272 bytes spill loads

2025-07-31 05:07:55.086464: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 484 bytes spill stores, 484 bytes spill loads

2025-07-31 05:07:55.378954: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 944 bytes spill stores, 944 bytes spill loads

2025-07-31 05:07:55.430067: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 988 bytes spill stores, 988 bytes spill loads

2025-07-31 05:07:55.556184: I external/l

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6485 - loss: 1.2088

2025-07-31 05:07:58.832068: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36', 272 bytes spill stores, 272 bytes spill loads

2025-07-31 05:07:58.838295: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36', 348 bytes spill stores, 348 bytes spill loads

2025-07-31 05:07:58.991838: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36', 816 bytes spill stores, 816 bytes spill loads

2025-07-31 05:07:59.132131: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36', 272 bytes spill stores, 272 bytes spill loads



1719/1719 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.6486 - loss: 1.2085 - val_accuracy: 0.9048 - val_loss: 0.3463
Epoch 2/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9032 - loss: 0.3459 - val_accuracy: 0.9194 - val_loss: 0.2810
Epoch 3/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9162 - loss: 0.2876 - val_accuracy: 0.9328 - val_loss: 0.2415
Epoch 4/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9288 - loss: 0.2511 - val_accuracy: 0.9398 - val_loss: 0.2141
Epoch 5/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9357 - loss: 0.2219 - val_accuracy: 0.9444 - val_loss: 0.1968
Epoch 6/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9422 - loss: 0.1977 - val_accuracy: 0.9472 - val_loss: 0.1818
Epoch 7/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9490 - loss: 0.1803 - val_accuracy: 0.9536 - val_loss: 0.1663
Epoch 8/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9510 - loss: 0.1709 - val_accura

In [28]:
model_version = '0002'
model_path = Path(model_name) / model_version
model.export(model_path)

INFO:tensorflow:Assets written to: my_mnist_model/0002/assets


INFO:tensorflow:Assets written to: my_mnist_model/0002/assets


Saved artifact at 'my_mnist_model/0002'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.uint8, name='keras_tensor_11')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  125864711264336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  125864711262032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  125864717768208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  125864717768016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  125864711261072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  125864711273168: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [29]:
sorted([str(path) for path in model_path.parent.glob('**/*')])

['my_mnist_model/0001',
 'my_mnist_model/0001/assets',
 'my_mnist_model/0001/fingerprint.pb',
 'my_mnist_model/0001/saved_model.pb',
 'my_mnist_model/0001/variables',
 'my_mnist_model/0001/variables/variables.data-00000-of-00001',
 'my_mnist_model/0001/variables/variables.index',
 'my_mnist_model/0002',
 'my_mnist_model/0002/assets',
 'my_mnist_model/0002/fingerprint.pb',
 'my_mnist_model/0002/saved_model.pb',
 'my_mnist_model/0002/variables',
 'my_mnist_model/0002/variables/variables.data-00000-of-00001',
 'my_mnist_model/0002/variables/variables.index']

**경고**: 텐서플로 서빙이 새 모델을 로드하기까지 잠시 기다려야 할 수 있다.

In [30]:
server_url = 'http://172.17.0.3:8501/v1/models/my_mnist_model:predict'
response = requests.post(server_url, request_json)
response.raise_for_status()
response = response.json()

In [31]:
response.keys()

dict_keys(['predictions'])

In [32]:
y_proba = np.array(response['predictions'])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.98, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]])